In [1]:
import pandas as pd
import numpy as np

# %% Imports
from pathlib import Path
import sys
root_dir = Path().resolve()
sys.path.append(str(root_dir / 'src'))

from common import *


In [2]:
root_dir.parent
meta=get_metadata()

In [3]:
sessions=get_sessions(True,.05,True,0.75)
sessions['impressions']=sessions['impressions'].str.split('\\|')
sessions['prices']=sessions['prices'].str.split('\\|')
print(sessions.dtypes)
sessions.head()

user_id                          object
session_id                       object
timestamp           datetime64[ns, UTC]
step                              int64
action_type                      object
reference                        object
platform                         object
city                             object
device                           object
current_filters                  object
impressions                      object
prices                           object
fake_split_train                   bool
is_train                           bool
dtype: object


,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,fake_split_train,is_train
0,004A07DM0IDW,1d688ec168932,2018-11-07 01:53:34+00:00,1,interaction item image,2059240,CO,"Santa Marta, Colombia",mobile,NaN,NaN,NaN,False,False
1,004A07DM0IDW,1d688ec168932,2018-11-07 01:53:34+00:00,2,interaction item image,2059240,CO,"Santa Marta, Colombia",mobile,NaN,NaN,NaN,False,False
2,004A07DM0IDW,1d688ec168932,2018-11-07 01:54:56+00:00,3,clickout item,1050068,CO,"Santa Marta, Colombia",mobile,NaN,"[2059240, 2033381, 1724779, 127131, 399441, 10...","[70, 46, 48, 76, 65, 65, 106, 66, 87, 43, 52, ...",False,False
3,004A07DM0IDW,1d688ec168932,2018-11-07 01:55:07+00:00,4,clickout item,1050068,CO,"Santa Marta, Colombia",mobile,NaN,"[2059240, 2033381, 1724779, 127131, 399441, 10...","[70, 46, 48, 76, 65, 65, 106, 66, 87, 43, 52, ...",False,False
4,004A07DM0IDW,1d688ec168932,2018-11-07 01:55:17+00:00,5,clickout item,1050068,CO,"Santa Marta, Colombia",mobile,NaN,"[2059240, 2033381, 1724779, 127131, 399441, 10...","[70, 46, 48, 76, 65, 65, 106, 66, 87, 43, 52, ...",False,False


# Get Splits

In [4]:
test = sessions.loc[sessions.is_train==False].reset_index(drop=True)
# train = sessions.loc[(sessions.is_train==True)]
train = sessions.loc[(sessions.is_train==True) & (sessions.fake_split_train==True)].reset_index(drop=True)
dev = sessions.loc[(sessions.is_train==True) & (sessions.fake_split_train==False)].reset_index(drop=True)

print('test',test.shape)
print('train',train.shape)
print('dev',dev.shape)

test (200069, 14)
train (537963, 14)
dev (155466, 14)


# Check number of test clickouts without a reference

In [5]:
nclickouts_nan = test.loc[(test.action_type=='clickout item') & (test.reference.isna()),'reference'].shape[0]
nclickouts = test.loc[(test.action_type=='clickout item'),'reference'].shape[0]
print('test clickouts without reference, total, and frac:', [nclickouts_nan,nclickouts,nclickouts_nan/nclickouts])

test clickouts without reference, total, and frac: [12856, 26345, 0.4879863351679636]


# Get Fake Test Set

In [6]:
dev_test,gt = process_fake_test(dev, frac_nan=0.488, seed=1234)
dev_test.head()
gt.head()

34    2795374
37    3143258
45    1236209
55    1175598
56      96234
Name: reference, dtype: object

# Munge Train to get training set.

In [40]:
dev_test.loc[(dev_test.reference.isnull()) | (dev_test.reference.str.contains('^[0-9]*$'))]


,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,fake_split_train,is_train
0,02SRUT1NQYH1,3599a6f709eab,2018-11-01 09:15:30+00:00,1,interaction item image,2795374,FI,"Krakow, Poland",mobile,NaN,NaN,NaN,False,True
1,02SRUT1NQYH1,3599a6f709eab,2018-11-01 09:15:30+00:00,2,interaction item image,2795374,FI,"Krakow, Poland",mobile,NaN,NaN,NaN,False,True
2,02SRUT1NQYH1,3599a6f709eab,2018-11-01 09:15:40+00:00,3,interaction item image,2795374,FI,"Krakow, Poland",mobile,NaN,NaN,NaN,False,True
3,02SRUT1NQYH1,3599a6f709eab,2018-11-01 09:15:40+00:00,4,interaction item image,2795374,FI,"Krakow, Poland",mobile,NaN,NaN,NaN,False,True
4,02SRUT1NQYH1,3599a6f709eab,2018-11-01 09:15:40+00:00,5,interaction item image,2795374,FI,"Krakow, Poland",mobile,NaN,NaN,NaN,False,True
5,02SRUT1NQYH1,3599a6f709eab,2018-11-01 09:15:50+00:00,6,interaction item image,2795374,FI,"Krakow, Poland",mobile,NaN,NaN,NaN,False,True
6,02SRUT1NQYH1,3599a6f709eab,2018-11-01 09:16:00+00:00,7,interaction item image,2795374,FI,"Krakow, Poland",mobile,NaN,NaN,NaN,False,True
7,02SRUT1NQYH1,3599a6f709eab,2018-11-01 09:16:00+00:00,8,interaction item image,2795374,FI,"Krakow, Poland",mobile,NaN,NaN,NaN,False,True
8,02SRUT1NQYH1,3599a6f709eab,2018-11-01 09:16:00+00:00,9,interaction item image,2795374,FI,"Krakow, Poland",mobile,NaN,NaN,NaN,False,True
9,02SRUT1NQYH1,3599a6f709eab,2018-11-01 09:16:10+00:00,10,interaction item image,2795374,FI,"Krakow, Poland",mobile,NaN,NaN,NaN,False,True
